## 2. Preparação para Tokenização (Pré-processamento)

Este notebook é responsável pelo pré-processamento dos dados do ENEM, incluindo a limpeza de texto, remoção de alternativas vazias, extração de gabaritos e distratores, aplicação de protocolos de normalização textual e preparação dos dados para etapas posteriores de vetorização e análise exploratória. O pré-processamento segue o fluxo:

* **Input**: CSV contendo todos os enunciados, alternativas e gabaritos

* **Output**: CSV com as colunas pré-processadas -> *numero_questao*, *enunciado*, *alternativas*,*nu_param_B*, *gabarito*, *ano*, *enunciado_limpo* e *alternativas_limpo*

In [ ]:
# Importando dependências para o pré-processamento dos dados
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords (palavras de parada)
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('portuguese'))

def clean_text(text):
    """Processando texto de acordo com o Protocolo Primi (2021), citado no artigo"""
    if not isinstance(text, str):  # Garantir que texto é string
        return ""
    
    words = re.findall(r'\b[a-zA-Zà-úÀ-ÚüÜ]+\b', text.lower())  # 1. Dividir item em palavras
    words = [word for word in words if word not in stop_words]  # 2. Remover stopwords
    return " ".join(sorted(set(words)))  # 3. Remover palavras duplicadas

def clean_alternatives(alt_text):
    """Processa as alternativas mantendo as letras (A:, B:, etc.) e limpando o conteúdo."""
    if not isinstance(alt_text, str):
        return ""
    
    alternatives = re.split(r'(?=[A-E]: )', alt_text)  # Divide mantendo os identificadores
    cleaned_alts = []
    
    for alt in alternatives:
        if ": " in alt:
            key, value = alt.split(": ", 1)  # Separa a letra da alternativa do conteúdo
            cleaned_value = clean_text(value)  # Aplica limpeza somente no conteúdo
            cleaned_alts.append(f"{key}: {cleaned_value}")  # Mantém o formato original
    
    return "; ".join(cleaned_alts)  # Junta novamente as alternativas


PADROES_DESCRICAO = [
    r"Descrição da Figura",
    r"Descrição da imagem",
    r"Descrição do mapa",
]

regex_descricao = r"(" + "|".join(PADROES_DESCRICAO) + r".*)"

def separar_descricao(row):
    alternativas = row['alternativas']
    
    match = re.search(regex_descricao, alternativas, re.IGNORECASE)
    
    if match:
        descricao = match.group(1).strip()
        alt_limpa = alternativas[:match.start()].strip()
        nova_questao = row['enunciado'].strip() + " " + descricao
        return pd.Series([alt_limpa, nova_questao])
    else:
        return pd.Series([alternativas, row['enunciado']])




def separate_distractors(row):
    items: list = [
        alternativa.strip()
        for alternativa in row["alternativas"].split(";")
        if alternativa.strip()
    ]

    mapping_items = {}
    for item in items:
        letra, texto = item.split(":", 1)
        mapping_items[letra.strip()] = texto.strip()

    # Extraindo texto do gabarito
    gabarito: str = row["gabarito"]
    row["gabarito_texto"] = mapping_items.get(gabarito.strip(), "")

    # Coletando alternativas distratoras
    row["distratores"] = "; ".join(
        [texto for letra, texto in mapping_items.items() if letra != gabarito]
    )

    return row

In [ ]:
# separa os casos onde as descrições estão nos enunciados

df[['alternativas', 'enunciado']] = df.apply(separar_descricao, axis=1)

In [ ]:
# separa os distratores

df = df.apply(separate_distractors, axis=1)

df.head()

In [ ]:
# Aplicando pré-processamento no enunciado e alternativas
df["enunciado_limpo"] = df["enunciado"].apply(clean_text)
df["alternativas_limpo"] = df["alternativas"].apply(clean_alternatives)
df["gabarito_texto_limpo"] = df["gabarito_texto"].apply(clean_text)
df["distratores_limpo"] = df["distratores"].apply(clean_text)

# Save cleaned CSV
df.to_csv("../data/final/cleaned_data.csv", index=False)
print("Processing complete. File saved as 'cleaned_data.csv'.")